In [ ]:
import pandas as pd
import numpy as np

In [ ]:
MONTHS_GEN = {
    1: "января",
    2: "февраля",
    3: "марта",
    4: "апреля",
    5: "мая",
    6: "июня",
    7: "июля",
    8: "августа",
    9: "сентября",
    10: "октября",
    11: "ноября",
    12: "декабря"
}

In [ ]:
data = pd.read_csv('weather_data.csv', sep=',').drop(['text'], axis='columns')

In [ ]:
data.head()

In [ ]:
month_list = list(MONTHS_GEN.values())
month_list.append('month')

In [ ]:
data = data.drop(month_list, axis='columns')
data_start = data.drop(['day_end'], axis='columns')

In [ ]:
data_start.head()

In [ ]:
data_start = data_start.drop(data_start[data_start['day_start'] > 31].index)
data_start = data_start.drop(data_start[data_start['day_start'] < 1].index)

In [ ]:
#способ повышения точности-удалить все мешки слов сумма которых больше 1 из датасета
# data_start = data_start.drop(data_start[data_start.iloc[:, 0:118].sum(axis = 1, skipna = True) > 3].index)

In [ ]:
start_x= data_start.drop('day_start', axis=1)
start_y = data_start['day_start'].astype('int')

In [ ]:
start_x.shape, start_y.shape

In [ ]:
start_y.head()

In [ ]:
start_y.describe()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
first_tree = DecisionTreeClassifier(random_state=17)

In [ ]:
from sklearn.model_selection import train_test_split
start_x_train, start_x_valid, start_y_train, start_y_valid = train_test_split(start_x, start_y, 
                                                      test_size=0.3, 
                                                      random_state=17)

In [ ]:
start_x_train.shape, start_x_valid.shape

In [ ]:
from sklearn.model_selection import  cross_val_score
cross_val_score(first_tree, start_x_train, start_y_train, cv=5)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
first_knn = KNeighborsClassifier()
np.mean(cross_val_score(first_knn, start_x_train, start_y_train, cv=5))

In [ ]:
%time
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(
    n_estimators=47, random_state=17,
    n_jobs=-1, max_features=2,
    )
rf.fit(start_x_train, start_y_train)

In [ ]:
%time
rf_valid_pred = rf.predict(start_x_valid)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(start_y_valid, rf_valid_pred)

In [ ]:
# from sklearn.metrics import confusion_matrix
# confusion_matrix(start_y_valid, rf_valid_pred)

In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
rfc = RandomForestClassifier(random_state=42, n_jobs=-1, oob_score=True)

In [ ]:
results = cross_val_score(rfc, start_x, start_y, cv=skf)

In [ ]:
print("CV accuracy score: {:.2f}%".format(results.mean()*100))

In [ ]:
# Инициализируем валидацию
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# Создаем списки для сохранения точности на тренировочном и тестовом датасете
train_acc = []
test_acc = []
temp_train_acc = []
temp_test_acc = []
trees_grid = range(90,100) 

In [ ]:
# Обучаем на тренировочном датасете
for ntrees in trees_grid:
    rfc = RandomForestClassifier(n_estimators=ntrees, random_state=42, n_jobs=-1, oob_score=True)
    temp_train_acc = []
    temp_test_acc = []
    for train_index, test_index in skf.split(start_x, start_y):
        X_train, X_test = start_x.iloc[train_index], start_x.iloc[test_index]
        y_train = start_y.iloc[train_index]
        y_test = start_y.iloc[test_index]
        rfc.fit(X_train, y_train)
        temp_train_acc.append(rfc.score(X_train, y_train))
        temp_test_acc.append(rfc.score(X_test, y_test))
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)

In [ ]:
train_acc, test_acc = np.asarray(train_acc), np.asarray(test_acc)
print("Best accuracy on CV is {:.2f}% with {} trees".format(max(test_acc.mean(axis=1))*100, 
                                                        trees_grid[np.argmax(test_acc.mean(axis=1))]))

In [ ]:
# Создаем списки для сохранения точности на тренировочном и тестовом датасете
train_acc = []
test_acc = []
temp_train_acc = []
temp_test_acc = []
max_depth_grid = range(80,120)

In [ ]:
# Обучаем на тренировочном датасете
for max_depth in max_depth_grid:
    rfc = RandomForestClassifier(n_estimators=91, random_state=42, n_jobs=-1, oob_score=True, max_depth=max_depth)
    temp_train_acc = []
    temp_test_acc = []
    for train_index, test_index in skf.split(start_x, start_y):
        X_train, X_test = start_x.iloc[train_index], start_x.iloc[test_index]
        y_train = start_y.iloc[train_index]
        y_test = start_y.iloc[test_index]
        rfc.fit(X_train, y_train)
        temp_train_acc.append(rfc.score(X_train, y_train))
        temp_test_acc.append(rfc.score(X_test, y_test))
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)
    
train_acc, test_acc = np.asarray(train_acc), np.asarray(test_acc)
print("Best accuracy on CV is {:.2f}% with {} max_depth".format(max(test_acc.mean(axis=1))*100, 
                                                        max_depth_grid[np.argmax(test_acc.mean(axis=1))]))

In [ ]:
# Создаем списки для сохранения точности на тренировочном и тестовом датасете
train_acc = []
test_acc = []
temp_train_acc = []
temp_test_acc = []
min_samples_leaf_grid = [1, 3, 5, 7, 9, 11, 13, 15, 17, 20, 22, 24]

In [ ]:
# Обучаем на тренировочном датасете
for min_samples_leaf in min_samples_leaf_grid:
    rfc = RandomForestClassifier(n_estimators=91, random_state=42, n_jobs=-1, 
                                 oob_score=True, min_samples_leaf=min_samples_leaf, max_depth=94)
    temp_train_acc = []
    temp_test_acc = []
    for train_index, test_index in skf.split(start_x, start_y):
        X_train, X_test = start_x.iloc[train_index], start_x.iloc[test_index]
        y_train = start_y.iloc[train_index]
        y_test = start_y.iloc[test_index]
        rfc.fit(X_train, y_train)
        temp_train_acc.append(rfc.score(X_train, y_train))
        temp_test_acc.append(rfc.score(X_test, y_test))
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)
    
train_acc, test_acc = np.asarray(train_acc), np.asarray(test_acc)
print("Best accuracy on CV is {:.2f}% with {} min_samples_leaf".format(max(test_acc.mean(axis=1))*100, 
                                                                min_samples_leaf_grid[np.argmax(test_acc.mean(axis=1))]))

In [ ]:
# Создаем списки для сохранения точности на тренировочном и тестовом датасете
train_acc = []
test_acc = []
temp_train_acc = []
temp_test_acc = []
max_features_grid = [2, 4, 6, 8, 10, 12, 14, 16]

# Обучаем на тренировочном датасете
for max_features in max_features_grid:
    rfc = RandomForestClassifier(n_estimators=91, random_state=42, n_jobs=-1, 
                                 oob_score=True, max_features=max_features)
    temp_train_acc = []
    temp_test_acc = []
    for train_index, test_index in skf.split(start_x, start_y):
        X_train, X_test = start_x.iloc[train_index], start_x.iloc[test_index]
        y_train = start_y.iloc[train_index]
        y_test = start_y.iloc[test_index]
        rfc.fit(X_train, y_train)
        temp_train_acc.append(rfc.score(X_train, y_train))
        temp_test_acc.append(rfc.score(X_test, y_test))
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)
    
train_acc, test_acc = np.asarray(train_acc), np.asarray(test_acc)
print("Best accuracy on CV is {:.2f}% with {} max_features".format(max(test_acc.mean(axis=1))*100, 
                                                        max_features_grid[np.argmax(test_acc.mean(axis=1))]))